# eidein

Interactive tool for dimensionality reduction of astronomical data and their visualisation.

In [ ]:
%matplotlib widget

import h5py
import ipywidgets
import numpy as np

from eidein import Eidein

from plot import plot_function

In [ ]:
with h5py.File("dataset.hdf5", "r") as datafile:
    ids = datafile["ids"][:]
    X = datafile["X"][:]
    uncertainty = datafile["uncertainty"][:]
    predictions = datafile["predictions"][:]
ids.dtype, X.dtype, uncertainty.dtype, predictions.dtype

def id2filename(plate, mjd, fiberid):
    return "spec-{:04d}-{}-{:04d}.fits".format(plate, mjd, fiberid)

vec_id2filename = np.vectorize(id2filename)
identifiers = vec_id2filename(ids[:, 0], ids[:, 1], ids[:, 2])

In [ ]:
label_widget = ipywidgets.FloatText(description="Redshift:", step=0.01)
eidein = Eidein(identifiers, X, predictions, uncertainty, plot_function, label_widget)
display(eidein)

In [ ]:
eidein.labelled